<a href="https://colab.research.google.com/github/JOSECOTA/CHALLENGES-DATA-SCIENCE/blob/main/CHALLENGE%20-%20ALURA%20IMOBILIARIA/SEMANA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> CHALLENGE - ALURA </center>
# <center> SEMANA 1 </center>

## Introdução

A imobiliária InsightPlaces, situada na cidade do Rio de Janeiro, está enfrentando dificuldades para alugar e vender imóveis. Em uma pesquisa de como empresas semelhantes operam no mercado, a InsightPlaces percebeu que esse problema pode estar relacionado aos valores dos imóveis e às recomendações que faz.

Dentro desse contexto, como podemos definir de forma eficiente os preços dos imóveis lidando com grande volume de dados? É importante recomendar imóveis utilizando outro critério? O que precisa ser feito?

Você faz parte do time de Ciência de Dados e Big Data da InsightPlaces e ficou responsável por auxiliar no processo de análise de dados dos imóveis localizados em alguns bairros da cidade do Rio de Janeiro.

Esse projeto tem algumas etapas como: ler e fazer o tratamento do histórico dos preços de imóveis no Rio de Janeiro, construir um modelo de regressão para precificar imóveis e criar um recomendador de imóveis. Para cada uma dessas etapas vamos utilizar a ferramenta PySpark que oferece uma melhor performance ao trabalharmos com grandes volumes de dados.

Criando sessão spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark


In [2]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Challenge-Imóveis') \
    .getOrCreate()

In [5]:
from pyspark.sql import functions as f

Montando o drive e carregando o dataset

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path = '/content/drive/MyDrive/Challenge-Imoveis/dataset_bruto.json'

In [8]:
dados_brutos = spark.read.json(path, multiLine=True)

## Explorar a base de dados utilizando recursos do PySpark

In [9]:
dados_brutos.show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+
|anuncio                                                               

In [10]:
dados_brutos.count()

89083

In [11]:
dados_brutos.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

## Transformar cada campo da coluna "anuncio" em uma coluna separada

Para nossa análise, apenas as informações do campo "anuncio" serão relevantes. Por isso, vamos focar em analisar as colunas desse campo.

In [12]:
anuncios = dados_brutos.select('anuncio')

In [13]:
anuncios.show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|anuncio                                                                                                                                                                                                                                                                                             |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{0, [], [16], [0], [], {Centro, 20061003, Rio de Janeiro, Rio de Janeiro, -22.906082, -43.18671, BR, Rua Buenos Ai

In [14]:
anuncios.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [15]:
anuncios = anuncios.withColumn('andar', anuncios.anuncio.andar)
anuncios = anuncios.withColumn('area_total', anuncios.anuncio.area_total)
anuncios = anuncios.withColumn('area_util', anuncios.anuncio.area_util)
anuncios = anuncios.withColumn('banheiros', anuncios.anuncio.banheiros)
anuncios = anuncios.withColumn('caracteristicas', anuncios.anuncio.caracteristicas)
anuncios = anuncios.withColumn('endereco', anuncios.anuncio.endereco)
anuncios = anuncios.withColumn('id', anuncios.anuncio.id)
anuncios = anuncios.withColumn('quartos', anuncios.anuncio.quartos)
anuncios = anuncios.withColumn('suites', anuncios.anuncio.suites)
anuncios = anuncios.withColumn('tipo_anuncio', anuncios.anuncio.tipo_anuncio)
anuncios = anuncios.withColumn('tipo_unidade', anuncios.anuncio.tipo_unidade)
anuncios = anuncios.withColumn('tipo_uso', anuncios.anuncio.tipo_uso)
anuncios = anuncios.withColumn('vaga', anuncios.anuncio.vaga)
anuncios = anuncios.withColumn('valores', anuncios.anuncio.valores)

In [16]:
anuncios = anuncios.drop('anuncio')

In [17]:
anuncios.show(5, truncate=False)

+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                        |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|0    |[]    

In [18]:
anuncios.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

## Filtrar a base de dados

O time de Data Science solicitou que fizéssemos alguns filtros nas colunas tipo_uso, tipo_unidade e tipo_anuncio da nossa base de dados:



*   tipo_uso: Residencial;
*   tipo_unidade: Apartamento;
*   tipo_anuncio: Usado.

Para entender o impacto que esses filtros terão sob a nossa base de dados, crie uma tabela de frequências para cada uma dessas colunas antes de filtrá-las.




In [19]:
freq_uso = anuncios.groupBy('tipo_uso').count()
freq_uso = freq_uso.withColumn('%',((freq_uso['count'])/(anuncios.count()))*100)
freq_uso.show()

+-----------+-----+-----------------+
|   tipo_uso|count|                %|
+-----------+-----+-----------------+
|  Comercial| 4542|5.098615897533761|
|Residencial|84541|94.90138410246624|
+-----------+-----+-----------------+



In [20]:
freq_unidade = anuncios.groupBy('tipo_unidade').count()
freq_unidade = freq_unidade.withColumn('%',((freq_unidade['count'])/(anuncios.count()))*100)
freq_unidade.show()

+------------+-----+------------------+
|tipo_unidade|count|                 %|
+------------+-----+------------------+
|      Outros|11963|13.429049313561512|
| Apartamento|66801| 74.98737132786279|
|        Casa|10319| 11.58357935857571|
+------------+-----+------------------+



In [21]:
freq_anuncio = anuncios.groupBy('tipo_anuncio').count()
freq_anuncio = freq_anuncio.withColumn('%',((freq_anuncio['count'])/(anuncios.count()))*100)
freq_anuncio.show()

+------------+-----+-------------------+
|tipo_anuncio|count|                  %|
+------------+-----+-------------------+
|       Usado|88827|  99.71262754958859|
|  Lançamento|  256|0.28737245041141407|
+------------+-----+-------------------+



In [22]:
anuncios = anuncios.select('*').where('tipo_uso == "Residencial"')
anuncios = anuncios.select('*').where('tipo_unidade == "Apartamento"')
anuncios = anuncios.select('*').where('tipo_anuncio == "Usado"')

In [23]:
anuncios.count()

66562

Com os filtros solicitados a base de dados teve uma diminuição de 89083 linhas para 66562.

## Transformar as colunas dos cômodos dos imóveis de listas para inteiros

Como pessoas engenheiras de dados, nós queremos entregar a base de dados de forma mais estruturada para que o time de ciência de dados possa realizar os tratamentos e criação do modelo de machine learning.

Nesse contexto, transforme os dados das colunas "quartos", "suites", "banheiros", "vaga", "area_total" e "area_util" de **listas** para **inteiros**.

In [24]:
anuncios.show(5)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [25]:
from pyspark.sql.types import IntegerType

In [26]:
anuncios = anuncios.withColumn('area_total',anuncios.area_total[0])\
                    .withColumn('area_util',anuncios.area_util[0])\
                    .withColumn('banheiros',anuncios.banheiros[0])\
                    .withColumn('quartos',anuncios.quartos[0])\
                    .withColumn('suites',anuncios.suites[0])\
                    .withColumn('vaga',anuncios.vaga[0])

In [27]:
anuncios.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [28]:
anuncios = anuncios.withColumn('area_total',anuncios.area_total.cast(IntegerType()))\
.withColumn('area_util',anuncios.area_util.cast(IntegerType()))\
.withColumn('banheiros',anuncios.banheiros.cast(IntegerType()))\
                   .withColumn('quartos',anuncios.quartos.cast(IntegerType()))\
                   .withColumn('suites',anuncios.suites.cast(IntegerType()))\
                   .withColumn('vaga',anuncios.vaga.cast(IntegerType()))\
                   .fillna(0)

In [29]:
anuncios.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 

In [30]:
anuncios.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|     0|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|     0|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [31]:
anuncios.filter(f.col("area_total")==0).count()

9203

In [32]:
anuncios.filter(f.col("area_util")==0).count()

0

In [33]:
anuncios.filter(f.col("banheiros")==0).count()

0

In [34]:
anuncios.filter(f.col("quartos")==0).count()

68

In [35]:
area_zerada = anuncios.filter((f.col("area_total")==0) & (f.col('area_util') == 0))
area_zerada.show()

+-----+----------+---------+---------+---------------+--------+---+-------+------+------------+------------+--------+----+-------+
|andar|area_total|area_util|banheiros|caracteristicas|endereco| id|quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso|vaga|valores|
+-----+----------+---------+---------+---------------+--------+---+-------+------+------------+------------+--------+----+-------+
+-----+----------+---------+---------+---------------+--------+---+-------+------+------------+------------+--------+----+-------+



## Tratamento de informações sobre localização

A equipe de ciência de dados nos solicitou que apenas as informações sobre **bairro** e **zona** da cidade fossem extraídas. 

Então, vamos analisar a coluna `endereco` e transformar apenas as informações sobre bairro e zona em colunas de nosso DataFrame.

In [36]:
anuncios.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 

In [37]:
anuncios = anuncios.withColumn('bairro', anuncios.endereco.bairro)
anuncios = anuncios.withColumn('zona', anuncios.endereco.zona)

In [38]:
anuncios = anuncios.drop('endereco')

In [39]:
anuncios.show(truncate=False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+------------------------+----------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |bairro                  |zona      |
+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------

## Transformar cada campo da coluna "valores" em uma coluna separada

Pensando em simplificar a compreensão dos dados para as pessoas cientistas de dados, vamos entender a estrutura da coluna **valores** que é a mais importante da base de dados.

```
root
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)
```

Podemos observar que os valores são do tipo array, ou seja, uma lista que contém elementos. Estes elementos são dicionários com as informações de `condominio`, `iptu`, `tipo`, e `valor`

Nesse contexto, **transforme os dados da coluna valores em colunas separadas**.

In [40]:
anuncios = anuncios.withColumn('condominio', anuncios.valores.condominio)
anuncios = anuncios.withColumn('iptu', anuncios.valores.iptu)
anuncios = anuncios.withColumn('tipo', anuncios.valores.tipo)
anuncios = anuncios.withColumn('valor', anuncios.valores.valor)

In [41]:
anuncios = anuncios.drop('valores')

In [42]:
anuncios.show(5, truncate=False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------+----------+----------+------+-------+-------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro   |zona      |condominio|iptu  |tipo   |valor  |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------+----------+----------+------+-------+-------+
|3    |43        |43       |1        |[Academia, 

In [43]:
from pyspark.sql.types import FloatType, StringType

In [44]:
anuncios = anuncios.withColumn('condominio',anuncios.condominio[0])\
                    .withColumn('iptu',anuncios.iptu[0])\
                    .withColumn('tipo',anuncios.tipo[0])\
                    .withColumn('valor',anuncios.valor[0])

In [45]:
anuncios = anuncios.withColumn('condominio',anuncios.condominio.cast(FloatType()))\
                    .withColumn('iptu',anuncios.iptu.cast(FloatType()))\
                    .withColumn('tipo',anuncios.tipo.cast(StringType()))\
                    .withColumn('valor',anuncios.valor.cast(FloatType()))\

In [46]:
anuncios = anuncios.fillna(0)

In [47]:
anuncios.show(5, truncate=False)

+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------+----------+----------+----+-----+-------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro   |zona      |condominio|iptu|tipo |valor  |
+-----+----------+---------+---------+-------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+---------+----------+----------+----+-----+-------+
|3    |43        |43       |1        |[Academia, Churrasqueir

## Tratamento para a coluna de valores

A **InsightPlaces** permite que o(a) anunciante crie um anúncio com duas opções de valor. Assim, o(a) cliente pode criar um anúncio que mostre tanto o valor de venda do imóvel quanto o seu valor de locação, juntamente com os valores de taxa de condomínio (quando houver) e taxa de IPTU. Estes valores são diferenciados pelo campo `tipo` que pode assumir os valores `Venda` e `Aluguel`.

Como se trata de um estudo sobre o preço de venda dos imóveis, o time de cientistas de dados solicitou apenas as informações do tipo VENDA.

Selecione apenas os valores de **Venda**.

---

**Dica:**

Vamos deixar aqui a documentação de dois métodos que podem ser úteis nessa etapa: 

* [pyspark.sql.functions.explode](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.explode.html)
* [pyspark.sql.DataFrame.join](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.join.html)

In [48]:
anuncios = anuncios.select('*').where('tipo == "Venda"')

In [49]:
anuncios.count()

66348

## Salvar os dados no formato parquet

In [50]:
anuncios.write.parquet('/content/drive/MyDrive/Challenge-Imoveis/anuncios.parquet')